In [33]:
# # #5/19 테스트
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Colab Notebooks/swishnet')
import sys
sys.path.append(os.getcwd())
!pip install import-ipynb

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
cutting_size = 32000 # 16000~48000 16000=1초
path = "./test/직접녹음.wav" #여기서 원하는 wav

In [35]:
import torch
import torchaudio
import import_ipynb

from model import swishnet

In [36]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


In [37]:
waveform, sample_rate = torchaudio.load(path)
print("원래 shape:", waveform.shape)

# 다채널이면 평균내서 모노로
if waveform.shape[0] > 1:
    waveform = waveform.mean(dim=0, keepdim=True)

# Resample
resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
resampled_waveform = resampler(waveform)

# 1채널이므로 squeeze로 1D 텐서 만들기
resampled_waveform = resampled_waveform.squeeze(0)
print(resampled_waveform.shape)
print("최종 shape:", resampled_waveform.shape)  # torch.Size([샘플 수])
print("샘플 수:", resampled_waveform.shape[0])
print("길이 (초):", resampled_waveform.shape[0] / 16000)

원래 shape: torch.Size([1, 2899968])
torch.Size([966656])
최종 shape: torch.Size([966656])
샘플 수: 966656
길이 (초): 60.416


In [38]:
model = swishnet(sample_rate = 16000).to(device)
#model.load_state_dict(torch.load('./swishnet_times_0_epoch_410(random).pth'))
model.load_state_dict(torch.load('./swishnet_times_0_epoch_410(random).pth', map_location=torch.device('cpu')))
model.eval()

swishnet(
  (dropout): Dropout(p=0.2, inplace=False)
  (mfcc_transform): MFCC(
    (amplitude_to_DB): AmplitudeToDB()
    (MelSpectrogram): MelSpectrogram(
      (spectrogram): Spectrogram()
      (mel_scale): MelScale()
    )
  )
  (block_1_up): swishnet_module(
    (tanh): Tanh()
    (sig): Sigmoid()
    (CausalConv1d): CausalConv1d(
      (conv): Conv1d(20, 16, kernel_size=(3,), stride=(1,))
    )
  )
  (block_1_down): swishnet_module(
    (tanh): Tanh()
    (sig): Sigmoid()
    (CausalConv1d): CausalConv1d(
      (conv): Conv1d(20, 16, kernel_size=(6,), stride=(1,))
    )
  )
  (block_2_up): swishnet_module(
    (tanh): Tanh()
    (sig): Sigmoid()
    (CausalConv1d): CausalConv1d(
      (conv): Conv1d(32, 8, kernel_size=(3,), stride=(1,))
    )
  )
  (block_2_down): swishnet_module(
    (tanh): Tanh()
    (sig): Sigmoid()
    (CausalConv1d): CausalConv1d(
      (conv): Conv1d(32, 8, kernel_size=(6,), stride=(1,))
    )
  )
  (block_3_up): swishnet_module(
    (tanh): Tanh()
    (si

In [39]:
def division(x):
    if x==0:
        return "car"
    if x==1:
        return "music"
    if x==2:
        return "speech"
    if x==3:
        return "noise"

In [40]:
num_segments = resampled_waveform.shape[0] // cutting_size
cut_list = []
t = 0
for i in range(num_segments):
    with torch.no_grad():
        cutted = resampled_waveform[i * cutting_size : (i + 1) * cutting_size]
        cut_list.append(cutted)
        cutted = cutted.unsqueeze(0).unsqueeze(0)
        pred = model(cutted.to(device))
        probs = torch.softmax(pred, dim=1)
    #car 0
    #music= 1
    #speech = 2
    #noise = 3
        preds = probs.argmax(dim=1)# 🔸예측 클래스 index
        print(preds,division(preds),t,"~",t+cutting_size/16000)
    t += cutting_size/16000

tensor([2]) speech 0 ~ 2.0
tensor([2]) speech 2.0 ~ 4.0
tensor([0]) car 4.0 ~ 6.0
tensor([2]) speech 6.0 ~ 8.0
tensor([2]) speech 8.0 ~ 10.0
tensor([2]) speech 10.0 ~ 12.0
tensor([0]) car 12.0 ~ 14.0
tensor([0]) car 14.0 ~ 16.0
tensor([0]) car 16.0 ~ 18.0
tensor([0]) car 18.0 ~ 20.0
tensor([0]) car 20.0 ~ 22.0
tensor([0]) car 22.0 ~ 24.0
tensor([0]) car 24.0 ~ 26.0
tensor([0]) car 26.0 ~ 28.0
tensor([0]) car 28.0 ~ 30.0
tensor([0]) car 30.0 ~ 32.0
tensor([0]) car 32.0 ~ 34.0
tensor([0]) car 34.0 ~ 36.0
tensor([0]) car 36.0 ~ 38.0
tensor([0]) car 38.0 ~ 40.0
tensor([0]) car 40.0 ~ 42.0
tensor([2]) speech 42.0 ~ 44.0
tensor([0]) car 44.0 ~ 46.0
tensor([0]) car 46.0 ~ 48.0
tensor([0]) car 48.0 ~ 50.0
tensor([0]) car 50.0 ~ 52.0
tensor([0]) car 52.0 ~ 54.0
tensor([3]) noise 54.0 ~ 56.0
tensor([0]) car 56.0 ~ 58.0
tensor([0]) car 58.0 ~ 60.0


In [45]:
from IPython.display import Audio

start_line = 6 # 0이면 0초 부터~cutsize 만큼

print(start_line*cutting_size/16000,"~",(start_line+1)*cutting_size/16000)
audio_array = cut_list[start_line].cpu().numpy()
sampling_rate = 16000

Audio(audio_array, rate=sampling_rate)

12.0 ~ 14.0
